# **Dependincies**

In [1]:
%%capture
!pip install polars
!pip install lightgbm==4.2.0
!pip install catboost==1.2.7
!pip install xgboost==2.0.3
!pip install optuna
!pip install lifelines

In [2]:
import shutil
import pandas as pd
import numpy as np
import re
import gc
import os
import math
import importlib
import random
import warnings
import pickle
import joblib
from tqdm import tqdm
import sys

from scipy.stats import skew, kurtosis
from collections import Counter,defaultdict
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import lightgbm as lgb
import catboost as cat, gc
import xgboost as xgb

print(lgb.__version__)
print(cat.__version__)
print(xgb.__version__)

from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.svm import SVR
from xgboost import XGBRegressor, XGBClassifier

import optuna

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import torch.nn.functional as F

from transformers import AutoModel,AutoTokenizer

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
sys.path.append('/notebooks/CIBMTR/utils')
sys.path.append('/notebooks/CIBMTR/train')

4.2.0
1.2.7
2.0.3


In [3]:
import tuning
import utils as my_utils
import metric
from metric import score
from utils import clr
import train as trainer
import config

lgb version:4.2.0
catboost version:1.2.7
xgboost version:2.0.3
lgb version:4.2.0
catboost version:1.2.7
xgboost version:2.0.3


# **LOAD DATA**

In [4]:
train_path = '/notebooks/CIBMTR/data/train.csv'
train_dic_path = '/notebooks/CIBMTR/data/data_dictionary.csv'

test_path = '/notebooks/CIBMTR/data/test.csv'
sub_path = '/notebooks/CIBMTR/data/sample_submission.csv'

In [5]:
train = pd.read_csv(train_path)
# dictionary_tow = pd.read_csv(train_dic_path)
dictionary = pd.read_csv('/notebooks/CIBMTR/output/dictionary.csv', encoding='cp932').drop('Unnamed: 0', axis=1)
test = pd.read_csv(test_path)
submission = pd.read_csv(sub_path)

In [6]:
TARGETS = ['efs_time', 'efs']

# **UM FEATURE ENGINNERING**

## **FE FUNCTIONS**

In [7]:
data=train
importlib.reload(my_utils)
importlib.reload(config)

<module 'config' from '/notebooks/CIBMTR/train/config.py'>

In [8]:
print(f'INITIAL DATA Shape:{data.shape}')
print('')
FE = my_utils.CIBMTR_FE(mode='train')
data, _ = FE.trans_obj_cate(data)

print('')
print(f'Final DATA Shape:{data.shape}')

INITIAL DATA Shape:(28800, 60)


Final DATA Shape:(28800, 60)


## **CONCAT OOF PREDICTIONS**

In [9]:
# ####
# # USING GDBT MODEL OOF
# ####
# EXP_NUM_LIST = ['EXP-39','EXP-40','EXP-43']
# predictions={}
# for EXP_NUM in EXP_NUM_LIST:
#     SAVE_PATH = f'{my_config.general.model_save_path}/TREE_{EXP_NUM}'
#     oof_preds = joblib.load(f'{SAVE_PATH}/oof_preds_{EXP_NUM}.pkl')
#     oof_preds = {key: np.clip(value, a_min=-1.0, a_max=1.0) for key, value in oof_preds.items()}
#     oof_preds = {model: oof_preds[model].flatten() for model in oof_preds}
    
#     cols = [f"{col.split('_')[0]}-{EXP_NUM}" for col in oof_preds]
#     oof_preds_df = pd.DataFrame(oof_preds)
#     oof_preds_df.columns = cols
#     predictions[EXP_NUM]=(oof_preds_df)

# predictions_df = pd.DataFrame()
# for col in predictions:
#     tmp_df = predictions[col]
#     predictions_df = pd.concat([predictions_df, tmp_df], axis=1)
# predictions_df = predictions_df.astype('float64')

# true = train['utility_agent1'].copy()
# print(len(predictions))

In [10]:
# ####
# # USING NN MODEL OOF
# ####

# # EXP_NUM_NN_LIST = 0
# # EXP_NUM_NN_LIST = ['EXP-50', 'EXP-68']
# EXP_NUM_NN_LIST = []
# if len(EXP_NUM_NN_LIST)!=0:
#     for EXP_NUM in EXP_NUM_NN_LIST:
#         SAVE_PATH = f'{my_config.general.model_save_path}/NN_{EXP_NUM}'
#         oof_preds = pd.read_csv(f'{SAVE_PATH}/oof.csv')
#         predictions_df[f'nn-{EXP_NUM}'] = oof_preds['pred']
# predictions_df.head(3)

# ####
# # USING DTNN MODEL OOF
# ####
# # EXP_NUM_NN_LIST = 0
# # EXP_NUM_NN_LIST = ['EXP-17']
# EXP_NUM_NN_LIST = []
# if len(EXP_NUM_NN_LIST)!=0:
#     for EXP_NUM in EXP_NUM_NN_LIST:
#         oof_preds = joblib.load(f'model/DTNN_{EXP_NUM}/oof_preds_{EXP_NUM}.pkl')
#         oof_preds = pd.DataFrame(oof_preds['dtnn'], columns=['pred'])
#         predictions_df[f'dtnn-{EXP_NUM}'] = oof_preds['pred']
# predictions_df.head(3)

In [11]:
# USE_OOF = False

# if USE_OOF:
#     data = pd.concat([data, predictions_df], axis=1)
#     # data = FE.fe6(data.copy())
#     # exp_num = 'EXP-35'
#     # data = FE.drop_cols_fi3(data.copy(), 
#     #                       save_path='model', 
#     #                       exp_num=f'TREE_{exp_num}', 
#     #                       topn=25)

# data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
# exp_num = 'EXP-48'
# data = FE.drop_cols_fi3(data.copy(), 
#                       save_path='model', 
#                       exp_num=f'TREE_{exp_num}', 
#                       topn=30)

# **OOF**

In [13]:
SPLIT = config.general.split
N_SPLITS=config.general.n_splits
SEED = config.general.seed
print(SPLIT)

kf


In [14]:
if SPLIT=='skf':
    kf = StratifiedKFold(n_splits=N_SPLITS, 
                     shuffle=True, 
                     random_state=SEED)
    for i, (_, val_index) in enumerate(kf.split(data, data.race_group)):
        data.loc[val_index, "fold"] = i
elif SPLIT=='kf':
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for i, (_, val_index) in enumerate(kf.split(data)):
        data.loc[val_index, "fold"] = i
data.head(3)

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,fold
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356,1.0
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672,1.0
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793,2.0


# **CHECK DATA**

In [15]:
FEATURES = data.drop(TARGETS+['fold']+['ID'], axis=1).columns.tolist()
len(FEATURES)

57

In [16]:
categorical_cols = data.drop(columns=TARGETS + ['ID']).select_dtypes(include=['object', 'category']).columns.to_list()
numerical_cols = data.drop(columns=TARGETS + ['ID']).select_dtypes(include=['number']).columns.to_list()

print(clr.RED + f'categorical_cols : {len(categorical_cols)}')
print(clr.RED + f'num_cols : {len(numerical_cols)}')

data[categorical_cols] = data[categorical_cols].astype(str).astype('category')
data = FE.create_target4(data)

categorical_cols : 35
num_cols : 23


# **TUENING HYPERPARAMETER**

In [17]:
importlib.reload(my_utils)
importlib.reload(metric)
importlib.reload(tuning)
importlib.reload(trainer)
importlib.reload(config)

lgb version:4.2.0
catboost version:1.2.7
xgboost version:2.0.3
lgb version:4.2.0
catboost version:1.2.7
xgboost version:2.0.3


<module 'config' from '/notebooks/CIBMTR/train/config.py'>

In [18]:
EXP_NUM='EXP-4'
MODEL_NAME = config.training.model_name
TARGET_TYPE = config.training.target_type
N_SPLITS=config.general.n_splits
SEED = config.general.seed
N_TRIALS = 30
SAVE_PATH = f'/notebooks/CIBMTR/output/{EXP_NUM}'

In [19]:
optuna=True
if optuna:
    if not os.path.exists(SAVE_PATH):
        os.makedirs(SAVE_PATH)
        print(f"Directory {SAVE_PATH} created.")
        
    shutil.copy('/notebooks/CIBMTR/train/config.py', f'{SAVE_PATH}/')
    joblib.dump(FEATURES, f'{SAVE_PATH}/features.pkl')
    joblib.dump(categorical_cols, f'{SAVE_PATH}/categorical_cols.pkl')
    joblib.dump(numerical_cols, f'{SAVE_PATH}/numerical_cols.pkl')
    
    tuning.optimize_params(
                    data=data,
                    features=FEATURES,
                    target_type=TARGET_TYPE,
                    n_split=N_SPLITS, 
                    cat_cols=categorical_cols,
                    cv_fold_func=np.average, 
                    model_name=MODEL_NAME,
                    n_trials=N_TRIALS,
                    exp=EXP_NUM,
                    seed=SEED)

[I 2025-01-14 08:01:08,656] A new study created in memory with name: Optimize boosting hyperparameters


Directory /notebooks/CIBMTR/output/EXP-4 created.
target_type:target4
0:	learn: -120215.1665527	test: -25959.1674451	best: -25959.1674451 (0)	total: 68.6ms	remaining: 2m 2s
500:	learn: -116559.4921719	test: -25419.7094147	best: -25419.7094147 (500)	total: 7.78s	remaining: 19.9s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25418.55674
bestIteration = 553

Shrink model to first 554 iterations.
C-index-American Indian or Alaska Native : 0.682
C-index-Asian : 0.681
C-index-Black or African-American : 0.655
C-index-More than one race : 0.691
C-index-Native Hawaiian or other Pacific Islander : 0.641
C-index-White : 0.660
Overall fold-0 C-index : 0.651
0:	learn: -120376.5406327	test: -25797.3755946	best: -25797.3755946 (0)	total: 21.1ms	remaining: 37.6s
500:	learn: -116688.9815912	test: -25215.9067369	best: -25215.6589741 (490)	total: 7.72s	remaining: 19.8s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25212.19869
bestIteration = 553

Shrink model 

[I 2025-01-14 08:02:34,196] Trial 0 finished with value: 0.6642397611231945 and parameters: {'n_estimators': 1784, 'learning_rate': 0.06383854273599891, 'max_depth': 6, 'colsample_bylevel': 0.44779316129262314, 'min_child_samples': 6, 'l2_leaf_reg': 0.07910069296318047, 'subsample': 0.08928361975502956}. Best is trial 0 with value: 0.6642397611231945.


C-index-White : 0.667
Overall fold-4 C-index : 0.664
----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6642397611231945
-[INFO] Trial All Fold(Log Loss): [0.6508499798233123, 0.6585237847574548, 0.669829163145366, 0.6778125168352156, 0.6641833610546235]
----------------------------------------------------------------------
target_type:target4
0:	learn: -120233.2566877	test: -25964.1249254	best: -25964.1249254 (0)	total: 21.7ms	remaining: 22.1s
500:	learn: -115764.3218927	test: -25402.7006897	best: -25400.0173117 (477)	total: 7.7s	remaining: 7.99s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25399.9849
bestIteration = 546

Shrink model to first 547 iterations.
C-index-American Indian or Alaska Native : 0.686
C-index-Asian : 0.681
C-index-Black or African-American : 0.659
C-index-More than one race : 0.692
C-index-Native Hawaiian or other Pacific Islander : 0.642
C-index-White : 0.664
Overall fold-0 C-index : 0.65

[I 2025-01-14 08:03:43,500] Trial 1 finished with value: 0.6650407082882447 and parameters: {'n_estimators': 1021, 'learning_rate': 0.050262316441024274, 'max_depth': 7, 'colsample_bylevel': 0.16832252378302714, 'min_child_samples': 13, 'l2_leaf_reg': 0.0682422576144196, 'subsample': 0.358619940509105}. Best is trial 1 with value: 0.6650407082882447.


...

----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6658934209602565
-[INFO] Trial All Fold(Log Loss): [0.6548983525889145, 0.6620223888263529, 0.6698378324988433, 0.6775740686720232, 0.6651344622151487]
----------------------------------------------------------------------
target_type:target4
0:	learn: -120217.6599663	test: -25960.5966431	best: -25960.5966431 (0)	total: 16.4ms	remaining: 23.4s
500:	learn: -116666.6339166	test: -25408.8413737	best: -25408.7046559 (452)	total: 6.58s	remaining: 12.2s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25405.05381
bestIteration = 595

Shrink model to first 596 iterations.
C-index-American Indian or Alaska Native : 0.684
C-index-Asian : 0.680
C-index-Black or African-American : 0.656
C-index-More than one race : 0.692
C-index-Native Hawaiian or other Pacific Islander : 0.637
C-index-White : 0.665
Overall fold-0 C-index : 0.65
0:	learn: -120388.5345386	test: -25800.3000294	be

[I 2025-01-14 08:38:24,422] Trial 28 finished with value: 0.6646725413555045 and parameters: {'n_estimators': 1427, 'learning_rate': 0.07651952696562564, 'max_depth': 7, 'colsample_bylevel': 0.08297480156812856, 'min_child_samples': 14, 'l2_leaf_reg': 0.057535963994654916, 'subsample': 0.06448457339536925}. Best is trial 17 with value: 0.6676909899034773.


----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6646725413555045
-[INFO] Trial All Fold(Log Loss): [0.6503632008043359, 0.6592421126469986, 0.6721735102648131, 0.6790916907687276, 0.6624921922926471]
----------------------------------------------------------------------
target_type:target4
0:	learn: -120186.8299139	test: -25954.1330224	best: -25954.1330224 (0)	total: 29ms	remaining: 47.1s
500:	learn: -116810.4845005	test: -25416.1779934	best: -25416.1779934 (500)	total: 7.2s	remaining: 16.1s
1000:	learn: -115947.1369431	test: -25412.3705335	best: -25406.8397373 (839)	total: 14.4s	remaining: 8.94s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25406.83974
bestIteration = 839

Shrink model to first 840 iterations.
C-index-American Indian or Alaska Native : 0.680
C-index-Asian : 0.683
C-index-Black or African-American : 0.656
C-index-More than one race : 0.693
C-index-Native Hawaiian or other Pacific Islander : 0.6

[I 2025-01-14 08:40:02,102] Trial 29 finished with value: 0.6659956039003869 and parameters: {'n_estimators': 1624, 'learning_rate': 0.06252653671671492, 'max_depth': 6, 'colsample_bylevel': 0.3764973265934654, 'min_child_samples': 11, 'l2_leaf_reg': 0.06462835217472966, 'subsample': 0.0932652257527633}. Best is trial 17 with value: 0.6676909899034773.


----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6659956039003869
-[INFO] Trial All Fold(Log Loss): [0.6523985475986567, 0.6611820965893083, 0.6694815095360717, 0.6800238589661483, 0.6668920068117495]
----------------------------------------------------------------------
----------------------------------------------------------------------
- Best Trial OOF: 0.6676909899034773
Best trial parameters:
    n_estimators: 1136
    learning_rate: 0.06731150536070069
    max_depth: 6
    colsample_bylevel: 0.6948788396823923
    min_child_samples: 13
    l2_leaf_reg: 0.061852557552403584
    subsample: 0.08358875397233823
Best parameters saved to /notebooks/CIBMTR/output/EXP-4/params.json
